# Инструкции Dockerfile: Синтаксис и семантика

## Введение

**Dockerfile** представляет собой текстовый документ, содержащий последовательность инструкций для автоматизированной сборки образа контейнера.

**Каждая инструкция** создаёт **отдельный слой** в файловой системе образа, что обеспечивает механизм кэширования и переиспользования промежуточных результатов сборки.

## Контекст сборки и файл `.dockerignore`

**Контекст сборки** — это набор файлов и каталогов, который передаётся демону Docker при выполнении `docker build`. По умолчанию контекстом является каталог, указанный как путь сборки (например, `.`). Все операции `COPY`/`ADD` работают **только** с файлами, попавшими в контекст.

**`.dockerignore`** определяет, какие пути следует исключить из контекста, чтобы:

* уменьшить размер передаваемого архива;
* снизить вероятность инвалидации кэша слоёв при незначительных изменениях;
* исключить из образа чувствительные данные.

Типичные записи: `.git`, кэши интерпретаторов, виртуальные окружения, крупные датасеты, временные каталоги.

## Механизм кэширования Docker

### Инструкции, создающие слои файловой системы

Docker создаёт новые слои файловой системы только для инструкций, которые модифицируют содержимое образа:

* RUN - создаёт слой (устанавливает пакеты, создаёт файлы, изменяет конфигурации)
* COPY - создаёт слой (добавляет файлы в образ)
* ADD - создаёт слой (добавляет файлы или распаковывает архивы)

### Инструкции, НЕ создающие слои файловой системы
Следующие инструкции только изменяют метаданные образа:

* CMD - записывает команду в метаданные образа
* ENTRYPOINT - записывает точку входа в метаданные образа
* ENV - устанавливает переменные окружения (технически создаёт слой, но пустой)
* EXPOSE - документирует порты в метаданных
* LABEL - добавляет метаданные
* USER - устанавливает пользователя в метаданных
* WORKDIR - устанавливает рабочую директорию (создаёт директорию, если не существует)

---

## Базовые инструкции Dockerfile

### FROM

**Синтаксис:**
```dockerfile
FROM <образ>[:<тег>] [AS <имя_этапа>]
```

**Назначение:**

Инструкция FROM определяет базовый образ, который служит основой для последующих операций сборки. Данная инструкция является обязательной и должна располагаться первой в Dockerfile (за исключением директивы parser и инструкции ARG для параметризации базового образа).

**Функциональность:**
- Инициализирует новый этап сборки образа
- Загружает указанный образ из реестра контейнеров (по умолчанию Docker Hub)
- Устанавливает базовую файловую систему для последующих модификаций

**Примеры использования:**
```dockerfile
FROM ubuntu:22.04
FROM python:3.11-slim
FROM scratch
FROM node:18-alpine AS builder
```

Образ `scratch` представляет собой специальный пустой образ, используемый для создания минималистичных контейнеров без базовой операционной системы.

---

### RUN

**Синтаксис:**
```dockerfile
RUN <команда>
RUN ["команда", "параметр1", "параметр2"]
```

**Назначение:**
Инструкция RUN выполняет произвольные команды в контексте создаваемого образа и фиксирует результат в виде нового слоя файловой системы.

**Функциональность:**
- Первая форма (shell form) выполняет команду через оболочку `/bin/sh -c` в системах Linux
- Вторая форма (exec form) выполняет команду напрямую без интерпретации оболочкой
- Результаты выполнения команды сохраняются в новом слое образа
- Каждая инструкция RUN создаёт отдельный слой, что влияет на итоговый размер образа

**Примеры использования:**
```dockerfile
RUN apt-get update && apt-get install -y \
    package1 \
    package2 \
    && rm -rf /var/lib/apt/lists/*

RUN ["/bin/bash", "-c", "echo hello"]
```

**Рекомендации по оптимизации:**

* Объединение нескольких команд в одну инструкцию RUN посредством операторов `&&` и `\` минимизирует количество слоёв и итоговый размер образа.

---

### CMD

**Синтаксис:**
```dockerfile
CMD ["исполняемый_файл", "параметр1", "параметр2"]
CMD команда параметр1 параметр2
CMD ["параметр1", "параметр2"]
```

**Назначение:**

Инструкция CMD определяет команду, которая будет выполнена при **запуске контейнера** из образа. В одном Dockerfile может присутствовать только одна инструкция CMD; при наличии нескольких инструкций учитывается исключительно последняя.

**Функциональность:**
- Определяет поведение контейнера по умолчанию
- Может быть переопределена аргументами командной строки при запуске контейнера
- В третьей форме предоставляет параметры по умолчанию для инструкции ENTRYPOINT

**Примеры использования:**
```dockerfile
FROM python:3.11

RUN pip install fastapi uvicorn
RUN mkdir /application
RUN useradd -m application_user

WORKDIR /application
COPY server.py /application/

CMD ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"]
```

**Запуск без аргументов:**

`docker run my-app`

Выполнится команда `uvicorn server:app --host 0.0.0.0 --port 8000`

**Запуск с переопределением команды:**

`docker run my-app python3 debug.py`

Выполнится команда `python3 debug.py`. CMD команда полностью заменяется

**Запуск с альтернативными параметрами:**

`docker run my-app uvicorn server:app --port 3000`

Выполнится: `uvicorn server:app --port 3000`. CMD полностью заменяется новой командой

В Dockerfile может быть указано несколько инструкций CMD, но будет использована только последняя.

```
FROM python:3.11

CMD ["echo", "first command"]
CMD ["echo", "second command"]
CMD ["echo", "third command"]
```

При запуске контейнера выполнится только: `echo third command`

---

### ENTRYPOINT

**Синтаксис:**
```dockerfile
ENTRYPOINT ["исполняемый_файл", "параметр1", "параметр2"]
ENTRYPOINT команда параметр1 параметр2
```

**Назначение:**
* Инструкция ENTRYPOINT конфигурирует контейнер для функционирования в качестве исполняемого файла.
* В отличие от CMD, команда ENTRYPOINT не может быть переопределена аргументами командной строки. Аргументы добавляются к указанной команде.

**Функциональность:**
- Определяет основной исполняемый процесс контейнера.
- Аргументы командной строки при запуске контейнера передаются в качестве параметров ENTRYPOINT.
- Обеспечивает предсказуемое поведение контейнера.

```dockerfile
FROM python:3.11

RUN pip install requests
RUN mkdir /application
RUN useradd -m application_user

WORKDIR /application
COPY data_processor.py /application/

ENTRYPOINT ["python", "data_processor.py"]
```

**Запуск без аргументов:**

`docker run my-processor`

Выполнится: `python data_processor.py`

**Запуск с аргументами:**

`docker run my-processor --input data.csv --output result.json`

`Выполнится: python data_processor.py --input data.csv --output result.json`. Аргументы добавляются к ENTRYPOINT

**Совместное использование CMD и ENTRYPOINT:**

```dockerfile
FROM python:3.11

RUN pip install fastapi
RUN mkdir /application
RUN useradd -m application_user

WORKDIR /application
COPY app.py /application/

ENTRYPOINT ["python", "app.py"]
CMD ["--host", "0.0.0.0", "--port", "5000", "--workers", "4"]
```

`docker run my-app`

Выполнится: `python app.py --host 0.0.0.0 --port 5000 --workers 4`

### ENTRYPOINT остаётся неизменным (`python app.py`), изменяются только параметры.

Аналогично **CMD**, в **Dockerfile** может присутствовать несколько инструкций **ENTRYPOINT**, но будет использована только последняя.

---

### COPY

**Синтаксис:**
```dockerfile
COPY [--chown=<пользователь>:<группа>] <источник> <назначение>
COPY [--chown=<пользователь>:<группа>] ["<источник>", "<назначение>"]
```

**Назначение:**
Инструкция COPY копирует файлы и директории из контекста сборки в файловую систему образа.

**Функциональность:**
- Переносит файлы из локальной файловой системы хоста в образ
- Поддерживает использование символов подстановки для указания множественных файлов
- Опция `--chown` устанавливает владельца скопированных файлов (только для систем Linux)
- Сохраняет метаданные файлов (разрешения, временные метки)

**Примеры использования:**
```dockerfile
COPY application.py /application/
COPY requirements.txt /application/requirements.txt
COPY --chown=1000:1000 config/ /etc/application/
COPY ["file with spaces.txt", "/destination/"]
```

---

### ADD

**Синтаксис:**
```dockerfile
ADD [--chown=<пользователь>:<группа>] <источник> <назначение>
ADD [--chown=<пользователь>:<группа>] ["<источник>", "<назначение>"]
```

**Назначение:**
Инструкция ADD расширяет функциональность COPY, предоставляя дополнительные возможности обработки источников.

**Функциональность:**
- Выполняет все операции инструкции COPY
- Автоматически извлекает локальные архивы tar в файловую систему образа
- Поддерживает загрузку файлов по URL (не рекомендуется для production)
- Определяет тип архива по содержимому, а не по расширению файла

**Примеры использования:**
```dockerfile
ADD archive.tar.gz /destination/
ADD https://example.com/file.txt /destination/
```

**Рекомендации:**
Официальная документация Docker рекомендует использовать COPY для простого копирования файлов, резервируя ADD исключительно для случаев, требующих автоматического извлечения архивов.

---

### WORKDIR

**Синтаксис:**
```dockerfile
WORKDIR /путь/к/директории
```

**Назначение:**
Инструкция WORKDIR устанавливает рабочую директорию для последующих инструкций `RUN`, `CMD`, `ENTRYPOINT`, `COPY` и `ADD`.

**Функциональность:**
- Создаёт указанную директорию, если она не существует
- Может использоваться многократно в Dockerfile
- Поддерживает относительные пути, которые интерпретируются относительно предыдущей инструкции `WORKDIR`
- Определяет директорию, в которой запускается процесс контейнера

**Примеры использования:**
```dockerfile
WORKDIR /application
WORKDIR source
WORKDIR subdir
```

В данном примере итоговым рабочим каталогом является `/application/source/subdir`.

---

### ENV

**Синтаксис:**
```dockerfile
ENV <ключ>=<значение>
ENV <ключ> <значение>
```

**Назначение:**
Инструкция ENV определяет переменные окружения, доступные в процессе сборки образа и во время выполнения контейнера.

**Функциональность:**
- Устанавливает постоянные переменные окружения
- Переменные доступны всем последующим инструкциям сборки
- Переменные сохраняются в образе и доступны при запуске контейнера
- Могут быть переопределены при запуске контейнера посредством флага `-e`

**Примеры использования:**
```dockerfile
ENV APPLICATION_PORT=8080
ENV PYTHON_VERSION=3.11
ENV PATH="/application/bin:${PATH}"
ENV DATABASE_HOST=localhost \
    DATABASE_PORT=5432 \
    DATABASE_NAME=production
```

---

### EXPOSE

**Синтаксис:**
```dockerfile
EXPOSE <порт>[/<протокол>]
```

**Назначение:**
Инструкция EXPOSE документирует сетевые порты, на которых контейнер ожидает входящие соединения во время выполнения.

**Функциональность:**
- Служит формой документации между разработчиком образа и оператором контейнера
- Не публикует порт автоматически
- По умолчанию предполагается протокол TCP; может быть явно указан UDP
- Фактическая публикация портов осуществляется посредством флага `-p` при запуске контейнера

**Примеры использования:**
```dockerfile
EXPOSE 80
EXPOSE 443
EXPOSE 8080/tcp
EXPOSE 53/udp
```

---

### VOLUME

**Синтаксис:**
```dockerfile
VOLUME ["/путь/к/директории"]
VOLUME /путь1 /путь2
```

**Назначение:**
Инструкция VOLUME создаёт точку монтирования для внешних томов или томов, совместно используемых другими контейнерами.

**Функциональность:**
- Определяет директории, предназначенные для хранения постоянных данных
- Данные в томах сохраняются независимо от жизненного цикла контейнера
- При запуске контейнера Docker создаёт анонимный том для каждой объявленной точки монтирования
- Изменения в томе после инструкции VOLUME в Dockerfile не сохраняются

**Примеры использования:**
```dockerfile
VOLUME ["/var/log"]
VOLUME /data
```

После создания контейнера данные находятся в служебной директории Docker:

`/var/lib/docker/volumes/<random-id>/_data`

---

### ARG

**Синтаксис:**
```dockerfile
ARG <имя>[=<значение_по_умолчанию>]
```

**Назначение:**
Инструкция ARG определяет переменные, которые пользователь может передать в процессе сборки образа посредством команды `docker build` с флагом `--build-arg`.

**Функциональность:**
- Переменные доступны только во время сборки образа
- Не сохраняются в финальном образе (в отличие от ENV)
- Могут иметь значения по умолчанию
- Область видимости ограничена этапом сборки, в котором они объявлены

**Примеры использования:**
```dockerfile
ARG VERSION=latest
ARG BUILD_DATE
ARG USER=application

FROM ubuntu:${VERSION}
RUN echo "Build date: ${BUILD_DATE}"
```

Использование при сборке:
```bash
docker build --build-arg VERSION=22.04 --build-arg BUILD_DATE=2024-01-01 .
```

---

### USER

**Синтаксис:**
```dockerfile
USER <пользователь>[:<группа>]
USER <UID>[:<GID>]
```

**Назначение:**
Инструкция USER устанавливает имя пользователя или UID, который будет использоваться при выполнении последующих инструкций RUN, CMD и ENTRYPOINT.

**Функциональность:**
- Изменяет контекст безопасности для выполнения команд
- Пользователь должен существовать в системе
- По умолчанию контейнеры выполняются от имени root
- Использование непривилегированного пользователя повышает безопасность контейнера

**Примеры использования:**
```dockerfile
RUN groupadd -r application && useradd -r -g application application
USER application
CMD ["python3", "application.py"]
```

---

### LABEL

**Синтаксис:**
```dockerfile
LABEL <ключ>=<значение> <ключ>=<значение> ...
```

**Назначение:**
Инструкция LABEL добавляет метаданные к образу в виде пар ключ-значение.

**Функциональность:**
- Предоставляет произвольные метаданные для документирования образа
- Не влияет на размер или производительность контейнера
- Может использоваться для версионирования, лицензирования, описания
- Поддерживает стандартные схемы аннотаций (например, OCI Image Spec)

```dockerfile
LABEL org.opencontainers.image.created="2024-10-07T10:00:00Z"
LABEL org.opencontainers.image.authors="developer@example.com"
LABEL org.opencontainers.image.url="https://example.com/project"
LABEL org.opencontainers.image.documentation="https://docs.example.com"
LABEL org.opencontainers.image.source="https://github.com/example/repository"
LABEL org.opencontainers.image.version="1.0.0"
LABEL org.opencontainers.image.revision="abc123def456"
LABEL org.opencontainers.image.vendor="Example Company"
LABEL org.opencontainers.image.licenses="MIT"
LABEL org.opencontainers.image.title="My Application"
LABEL org.opencontainers.image.description="Application for data processing"
```

**Примеры использования:**
```dockerfile
LABEL version="1.0"
LABEL maintainer="developer@example.com"
LABEL description="Application container for production deployment"
LABEL org.opencontainers.image.source="https://github.com/example/repository"
```

---

## Порядок выполнения инструкций

Docker обрабатывает инструкции Dockerfile последовательно, сверху вниз. Каждая инструкция создаёт промежуточный слой образа, который кэшируется для ускорения последующих сборок. Изменение любой инструкции инвалидирует кэш для данной инструкции и всех последующих.

---

## Базовые команды Docker для работы с контейнерами

### Команда `docker build`

```bash
docker build [опции] <путь_к_контексту_сборки>
```

**Флаг -t**

Задаёт имя и тег образа.

```bash
docker build -t <имя_образа>:<тег> .
```

---

### Команда `docker run`

```bash
docker run [опции] <имя_образа>:<тег> [команда]
```

**Флаг -v (volume)**

Cвязывает директорию хоста с директорией контейнера.

```bash
docker run -v <путь_на_хосте>:<путь_в_контейнере> <образ>
```

---

## Самостоятельная работа

1. Модифицируйте скрипт, добавьте отслеживание обучения(MLFlow/ClearML)

2. Создайте `Dockerfile`

* Используйте базовый образ `python:3.11-slim`
* Используйте `/app/models` для хранения результатов
* Скрипт обучения при запускается при старте контейнера

3. Добавьте постоянное хранилище(на хосте), где будут сохраняться метрики и веса модели

## Опционально

4. Параметризация обучения

    Модифицируйте `train_model.py` для приёма параметров через переменные окружения:

    * `N_ESTIMATORS` - количество деревьев (по умолчанию 100)
    * `TEST_SIZE` - размер тестовой выборки (по умолчанию 0.3)
    * `RANDOM_STATE` - seed для воспроизводимости (по умолчанию 42)